In [1]:
import findspark
findspark.init("/usr/local/bin/spark-3.2.3-bin-hadoop3.2")
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

23/03/20 21:29:42 WARN Utils: Your hostname, funicello-xps-9315 resolves to a loopback address: 127.0.1.1; using 172.27.75.183 instead (on interface wlp0s20f3)
23/03/20 21:29:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/20 21:29:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Data to adjacency list

In [2]:
# the dataset is composed by Baby items, Digital Music and Digital Videogames
# RDD (customer,product)
tsv0 = spark.read.option("header", "true").option("sep", "\t").csv("amazon_reviews_us_Baby_v1_00.tsv").select(["customer_id","product_parent"]).rdd
tsv1 = spark.read.option("header", "true").option("sep", "\t").csv("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv").select(["customer_id","product_parent"]).rdd
tsv2 = spark.read.option("header", "true").option("sep", "\t").csv("amazon_reviews_us_Digital_Video_Games_v1_00.tsv").select(["customer_id","product_parent"]).rdd
tsv = tsv0.union(tsv1).union(tsv2)
data = tsv
print(f"Rows in the dataset: {data.count()}")
data.take(1)

Rows in the dataset: 3587247


[Row(customer_id='9970739', product_parent='329991347')]

In [3]:
data.take(2)

[Row(customer_id='9970739', product_parent='329991347'),
 Row(customer_id='23538442', product_parent='646108902')]

In [4]:
# Extract id of Digital Videogames products in the database
video_games = tsv2.map(lambda x: x[1]).distinct()
video_games.take(2)

['603406193', '452532551']

In [21]:
# Extract id of Baby products in the database
baby = tsv0.map(lambda x: x[1]).distinct()
baby.take(2)

['93827401', '67911244']

In [22]:
# Extract id of Digital Music products in the database
music = tsv1.map(lambda x: x[1]).distinct()
music.take(2)

['165995375', '137422612']

In [5]:
#### CELL 5
# Create couples of items purchased by the same customer
# RDD [(customer1, (product1,product2)),(customer2, (product3,product4)),(...)]
data_joined = data.join(data)
data_joined.take(2)

[('25720064', ('104910197', '104910197')),
 ('28727683', ('43024790', '43024790'))]

In [6]:
#### CELL 6
# Remove duplicate item couples created by the join
# RDD [(customer1, (product1,product2)),(customer2, (product3,product4)),(...)]
data_filtered = data_joined.filter(lambda x : x[1][0]!= x[1][1])
data_filtered.take(2)

[('28727683', ('43024790', '752080849')),
 ('28727683', ('752080849', '43024790'))]

In [7]:
#### CELL 7
# Create list of links between products
# RDD [(product1,product2),(product3,product4),(...))]
edge_list = data_filtered.map(lambda x : x[1])
edge_list.take(2) 

[('43024790', '752080849'), ('752080849', '43024790')]

In [8]:
#### CELL 8
# Make second item a list
# RDD [(node1, [node2]),(node1, [node2]),(...)]
adjacency1 = edge_list.mapValues(lambda v: [v])

# RDD [(node1,[node2,node3,...]),(...)]
adjacency_list = adjacency1.reduceByKey(lambda a,b: a + b)

adjacency_list.take(1)

[('439716423',
  ['543525459',
   '496365324',
   '419902739',
   '235995163',
   '231583317',
   '716235929',
   '81415061',
   '354873430',
   '65780667',
   '944771005',
   '995454389',
   '376052348',
   '462322021',
   '936800356',
   '905916714',
   '70476958',
   '770074498',
   '929801740',
   '877861502',
   '513427451',
   '65006601',
   '2143250',
   '561545515',
   '891725593',
   '591175200',
   '653529573',
   '294256847',
   '704914555',
   '874902758',
   '695278058',
   '24467264',
   '213512194',
   '757238672',
   '67653417',
   '973007296',
   '537317100',
   '720285389',
   '220957025',
   '55572336',
   '62898823',
   '386222597',
   '712747703',
   '700402133',
   '57839336',
   '563075360',
   '298076625',
   '606841120',
   '808924129',
   '378583797',
   '844031020',
   '712747703',
   '611497019',
   '895585963',
   '578210015',
   '995735900',
   '744785973',
   '240619615',
   '648533854',
   '162532204',
   '106247650',
   '431024885',
   '404815841',
   '

### Calculate pagerank

In [9]:
#### CELL 9
def calculate_initial_contribs(row):
    """
    This function takes elements from the joined dataset above and
    computes the contribution to each outgoing link based on the
    current rank.
    """
    item, array = row
    num_edges = len(array)
    for node in array:
        yield item, node, 1/num_edges

# transition matrix stored as triplets
# RDD [(product1,(product2,probability)),(product3,(product4,probability))]
trans_matrix = adjacency_list.flatMap(calculate_initial_contribs).cache()
trans_matrix.take(10) 

23/03/20 21:30:39 WARN BlockManager: Task 202 already completed, not releasing lock for rdd_70_0


[('439716423', '543525459', 0.0010204081632653062),
 ('439716423', '496365324', 0.0010204081632653062),
 ('439716423', '419902739', 0.0010204081632653062),
 ('439716423', '235995163', 0.0010204081632653062),
 ('439716423', '231583317', 0.0010204081632653062),
 ('439716423', '716235929', 0.0010204081632653062),
 ('439716423', '81415061', 0.0010204081632653062),
 ('439716423', '354873430', 0.0010204081632653062),
 ('439716423', '65780667', 0.0010204081632653062),
 ('439716423', '944771005', 0.0010204081632653062)]

In [10]:
# Product names
keys = trans_matrix.map(lambda x : x[0]).distinct().collect()
# map product names to integers
indexmap = dict(zip(keys, range(len(keys))))

# map items to indexes
# RDD [(index_product1,(index_product2,probability)),(index_product3,(index_product4,probability))]
hashed_matrix = trans_matrix.map(lambda x: (indexmap[x[0]],(indexmap[x[1]],x[2])))
hashed_matrix.take(2)

23/03/20 21:30:54 WARN BlockManager: Task 347 already completed, not releasing lock for rdd_70_0


[(0, (569322, 0.0010204081632653062)), (0, (471457, 0.0010204081632653062))]

In [14]:
import numpy as np

# Array of pagerank stored in memory
n = len(keys) # num of products in adjacency list
pgrnk = np.ones(n)/n # time t
previus_pgrnk = np.ones(n) # time -1
     
# MSE distance between previous pgrank iteration
def l2distance(v, q):   
    return sum([(q_el - v_el)**2 for v_el, q_el in zip(v, q)])

In [15]:
#### CELL 15
tolerance = 10e-10
max_iterations = 300
i = 0
tax = 0.85

while(l2distance(previus_pgrnk, pgrnk) >= tolerance and i < max_iterations):
    previus_pgrnk = np.copy(pgrnk)

    page_rank_values = (hashed_matrix
                    .mapValues(lambda v: tax*(v[1]*pgrnk[v[0]]) + (1-tax)*(1/n)) 
                    .reduceByKey(lambda a, b: a+b)
                    .sortByKey()
                    .collect()
                    )
  
    pgrnk = np.array([c for (i, c) in page_rank_values])
    i=i+1

In [46]:
# Avg pagerank
np.mean(pgrnk)

0.001135444161043827

### Topic aware page rank

In [44]:
vgdict = video_games.collect()
vg_array = [1 if (x in vgdict) else 0 for x in indexmap]

In [50]:
# Reset arrays
n = len(keys) # num of products in adjacency list
pgrnk = np.ones(n)/n # time t
previus_pgrnk = np.ones(n) # time -1


In [ ]:
#### CELL 18
tolerance = 10e-10
max_iterations = 300
i = 0
tax = 0.85

# size of the page in the topic set
len_topic = video_games.count()    

# if the page is in the selected topic set it's pgrank calculation will have
# the added weight from the teletrasportation
def calcPgRnk(row):
    return tax*(row[1]*pgrnk[row[0]]) + vg_array[row[0]]*(1-tax)*(1/len_topic)


while(l2distance(previus_pgrnk, pgrnk) >= tolerance or i < max_iterations):
  
    previus_pgrnk = np.copy(pgrnk) 
    page_rank_values = (hashed_matrix
                        .mapValues(calcPgRnk) 
                        .reduceByKey(lambda a, b: a+b) 
                        .sortByKey()
                        .collect()
                       )
  
    pgrnk = np.array([c for (i, c) in page_rank_values])
    i = i + 1

In [57]:
# Avg pgrnk for videogame products
sum = 0
for pg,vg in zip(pgrnk,vg_array):
    sum = pg*vg + sum 

sum/len_topic

0.007904045707260303

In [58]:
# Avg pgrnk for products in the other two categories
sum_others = 0
for pg,vg in zip(pgrnk,vg_array):
    sum_others = pg*int(not(vg)) + sum_others

sum_others / (len(pgrnk)-len_topic)

0.0005901374609417463